In [1]:
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
DATA_DIR = "/Users/rishavghosh/Desktop/python/Oasis_dataset"

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),               # Resize smaller edge to 256
    transforms.CenterCrop(224),           # Crop to 224x224 as VGG expects
    transforms.ToTensor(),                # Convert PIL Image to Torch tensor
    transforms.Normalize(                 # Normalize to ImageNet mean/std
        mean=[0.485, 0.456, 0.406],      
        std=[0.229, 0.224, 0.225]
    )
])

In [4]:
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
print("Classes:", full_dataset.classes)

Classes: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']


In [5]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

Train size: 96336
Test size: 24085


In [6]:
from torch.utils.data import WeightedRandomSampler
import numpy as np
import torch

# Extract labels from train dataset
train_targets = [full_dataset.samples[i][1] for i in train_dataset.indices]
class_counts = np.bincount(train_targets)

print("Train class counts:", dict(zip(full_dataset.classes, class_counts)))

# Compute class weights
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = [class_weights[label] for label in train_targets]

# Sampler for balanced training
train_sampler = WeightedRandomSampler(weights=sample_weights,
                                      num_samples=len(sample_weights),
                                      replacement=True)


Train class counts: {'Mild Dementia': 11116, 'Moderate Dementia': 5580, 'Non Demented': 61551, 'Very mild Dementia': 18089}


In [7]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
import torchvision.models as models
model = models.vgg16(pretrained=True)

/Users/rishavghosh/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rishavghosh/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
for param in model.features.parameters():
    param.requires_grad = False

In [10]:
from torchinfo import summary
summary(model = model,
        input_size = (32, 3, 224, 224),
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
       )

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Sequential (features)                  [32, 3, 224, 224]    [32, 512, 7, 7]      --                   False
│    └─Conv2d (0)                        [32, 3, 224, 224]    [32, 64, 224, 224]   (1,792)              False
│    └─ReLU (1)                          [32, 64, 224, 224]   [32, 64, 224, 224]   --                   --
│    └─Conv2d (2)                        [32, 64, 224, 224]   [32, 64, 224, 224]   (36,928)             False
│    └─ReLU (3)                          [32, 64, 224, 224]   [32, 64, 224, 224]   --                   --
│    └─MaxPool2d (4)                     [32, 64, 224, 224]   [32, 64, 112, 112]   --                   --
│    └─Conv2d (5)                        [32, 64, 112, 112]   [32, 128, 112, 112]  (73,856)             False
│    └─ReLU (

In [11]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [12]:
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 4)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [13]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [14]:
import time
num_epochs = 5

for epoch in range(num_epochs):
    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, test_loader, criterion, device)

    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs}, Time: {elapsed:.2f}s")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss  : {val_loss:.4f}, Val Acc  : {val_acc:.4f}")

Epoch 1/5, Time: 3202.83s
  Train Loss: 0.3480, Train Acc: 0.8566
  Val Loss  : 0.2297, Val Acc  : 0.9059
Epoch 2/5, Time: 3287.28s
  Train Loss: 0.1453, Train Acc: 0.9462
  Val Loss  : 0.1725, Val Acc  : 0.9303
Epoch 3/5, Time: 3555.69s
  Train Loss: 0.1005, Train Acc: 0.9628
  Val Loss  : 0.1072, Val Acc  : 0.9586
Epoch 4/5, Time: 3450.24s
  Train Loss: 0.0798, Train Acc: 0.9712
  Val Loss  : 0.1443, Val Acc  : 0.9466
Epoch 5/5, Time: 3157.36s
  Train Loss: 0.0647, Train Acc: 0.9775
  Val Loss  : 0.1038, Val Acc  : 0.9625


In [15]:
torch.save(model.state_dict(), 'vgg16_alzheimers_extraData.pth')